In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split

from keras.layers import Input, Dense
from keras.models import Model

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray

In [2]:
np.random.seed(1120)

In [3]:
data = pd.read_csv('./data/AT.dat', delimiter="\t",
                   skiprows=[0], names=["k","r","x","A[k,r,x]"])
print(data.head())
data = data.to_numpy()

# Use first 8,056 points as training/validation and rest as test set.
train_data = data[0:8056]
test_data = data[8056:]

       k         r         x  A[k,r,x]
0  0.001  0.010817 -0.973907  1.009217
1  0.001  0.010817 -0.679410  1.009143
2  0.001  0.010817 -0.148874  1.009077
3  0.001  0.010817  0.148874  1.009077
4  0.001  0.010817  0.679410  1.009142


### Implementation
Now, we have a fair understanding of what GA is and how it works. Next, let’s get to coding.

We will use wind power forecast data, which is available at the following link. It consists of normalized (between zero and one) wind power measurements from seven wind farms. To keep things simple, we will use first wind farm data (column named wp1) but I encourage the reader to experiment and extend the code to forecast energy for all seven, wind farms.

Let’s import required packages, load the dataset and define two helper functions. The first method prepare_dataset will segment the data into chunks to create X, Y pair for model training. The X will the wind power values from the past (e.g. 1 to t-1) and Y will be future value at time t. The second method train_evaluate perform three things, 1) decoding GA solution to get window size and number of units. 2) Prepare the dataset using window size found by GA and divide into train and validation set, and 3) train LSTM model, calculate RMSE on validation set and return it as a fitness score of the current GA solution.

In [4]:
data[:,1].shape
data[:,0:3].shape[1]

3

In [5]:
np.zeros(11)[6:11]

array([0., 0., 0., 0., 0.])

In [73]:
def prepare_dataset(data):
    X, Y = np.empty((0)), np.empty((0))
    X = data[:,0:3]
    Y = data[:,3]  
    return X, Y

def train_evaluate(ga_individual_solution):   
    t = time.time()
    
    # Decode GA solution to integer for window_size and num_units
    deep_size_bits = BitArray(ga_individual_solution[0:3])
    num_units_bits = BitArray(ga_individual_solution[3:8])
    learning_rate_bits = BitArray(ga_individual_solution[8:12])
    batch_size_bits = BitArray(ga_individual_solution[12:14])
    epochs_bits = BitArray(ga_individual_solution[14:])
    
    deep_size = 2*deep_size_bits.uint +2
    num_units = 2*num_units_bits.uint +1
    learning_rate = (2*learning_rate_bits.uint+1)*10**(-4)
    batch_size = 2**(batch_size_bits.uint +1)
    epochs = (epochs_bits.uint +1)*50
    
    print('\nDeep Size: ', deep_size, ', Num of Units: ', num_units, ', Learning rate: ', learning_rate)
    print('Batch Size: ', batch_size, ", Num of Epochs: ", epochs)
    
    # Segment the train_data based on new window_size; split into train and validation (80/20)
    X,Y = prepare_dataset(train_data)
    X_train, X_test, y_train, y_test = split(X, Y, test_size = 0.20, random_state = 1120)
    
    # Train LSTM model and predict on validation set
    model = keras.Sequential()
    model.add(keras.Input(shape=(int(X_train.shape[1]),)))
    model.add(layers.Dense(num_units, input_shape=(int(X_train.shape[1]),)))
#     x = LSTM(num_units, input_shape=(window_size,1))(inputs)
    
    for i in range(deep_size):        
        model.add(layers.Dense(num_units, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0,
                                   patience=15,
                                   restore_best_weights=True)]
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=epochs, callbacks=callbacks, batch_size=batch_size, shuffle=True)
    y_pred = model.predict(X_test)
    
    # Calculate the RMSE score as fitness score for GA
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('Validation RMSE: ', rmse)
    print('Time passed: ', time.time()-t)
    
    return rmse,

A continuación, use la paquetería DEAP para definir las cosas para ejecutar GA. Usaremos una representación binaria para la solución de longitud diez. Se inicializará aleatoriamente utilizando la distribución de Bernoulli. Del mismo modo, se utiliza el crossover ordenado, la mutación aleatoria y la selección de la rueda de la ruleta. Los valores del parámetro GA se inicializan arbitrariamente; Te sugerimos que juegues con diferentes configuraciones.



In [ ]:
population_size = 15
num_generations = 6
gene_length = 16

# As we are trying to minimize the RMSE score, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0
creator.create('FitnessMax', base.Fitness, weights = (-1.0,))
creator.create('Individual', list , fitness = creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, 
n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)

population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, 
                        ngen = num_generations, verbose = True)


Deep Size:  14 , Num of Units:  35 , Learning rate:  0.0001
Batch Size:  8 , Num of Epochs:  50
Train on 6444 samples, validate on 1612 samples
Epoch 1/50
6444/6444 [==============================] - 14s 2ms/sample - loss: 0.5063 - val_loss: 0.0656
Epoch 2/50
6444/6444 [==============================] - 8s 1ms/sample - loss: 0.0121 - val_loss: 0.0018
Epoch 3/50
6444/6444 [==============================] - 8s 1ms/sample - loss: 0.0021 - val_loss: 0.0046
Epoch 4/50
6444/6444 [==============================] - 8s 1ms/sample - loss: 0.0014 - val_loss: 0.0030
Epoch 5/50
6444/6444 [==============================] - 8s 1ms/sample - loss: 0.0048 - val_loss: 8.8550e-04
Epoch 6/50
6444/6444 [==============================] - 9s 1ms/sample - loss: 0.0015 - val_loss: 7.6060e-04
Epoch 7/50
6444/6444 [==============================] - 7s 1ms/sample - loss: 0.0055 - val_loss: 8.3111e-04
Epoch 8/50
6444/6444 [==============================] - 8s 1ms/sample - loss: 6.5142e-04 - val_loss: 5.7622e-04
Ep

Epoch 21/50
6444/6444 [==============================] - 7s 1ms/sample - loss: 2.8552e-04 - val_loss: 2.5563e-04
Epoch 22/50
6444/6444 [==============================] - 10s 2ms/sample - loss: 2.7530e-04 - val_loss: 2.3150e-04
Epoch 23/50
6444/6444 [==============================] - 9s 1ms/sample - loss: 2.5900e-04 - val_loss: 2.1037e-04
Epoch 24/50
6444/6444 [==============================] - 7s 1ms/sample - loss: 2.4768e-04 - val_loss: 2.3064e-04
Epoch 25/50
6444/6444 [==============================] - 6s 988us/sample - loss: 2.4731e-04 - val_loss: 2.3280e-04
Epoch 26/50
6444/6444 [==============================] - 7s 1ms/sample - loss: 2.5685e-04 - val_loss: 2.7054e-04
Epoch 27/50
6444/6444 [==============================] - 7s 1ms/sample - loss: 2.5573e-04 - val_loss: 2.1188e-04
Epoch 28/50
6444/6444 [==============================] - 7s 1ms/sample - loss: 2.5362e-04 - val_loss: 3.8112e-04
Epoch 29/50
6444/6444 [==============================] - 7s 1ms/sample - loss: 2.7012e-04 - v

Epoch 41/150
6444/6444 [==============================] - 5s 749us/sample - loss: 2.2211e-04 - val_loss: 1.9990e-04
Epoch 42/150
6444/6444 [==============================] - 4s 612us/sample - loss: 2.6370e-04 - val_loss: 3.8105e-04
Epoch 43/150
6444/6444 [==============================] - 3s 536us/sample - loss: 2.2038e-04 - val_loss: 2.3806e-04
Epoch 44/150
6444/6444 [==============================] - 3s 537us/sample - loss: 2.3811e-04 - val_loss: 2.1136e-04
Epoch 45/150
6444/6444 [==============================] - 3s 541us/sample - loss: 2.3375e-04 - val_loss: 2.0948e-04
Epoch 46/150
6444/6444 [==============================] - 3s 538us/sample - loss: 2.2532e-04 - val_loss: 2.3361e-04
Epoch 47/150
6444/6444 [==============================] - 4s 675us/sample - loss: 2.2108e-04 - val_loss: 2.4464e-04
Epoch 48/150
6444/6444 [==============================] - 4s 589us/sample - loss: 2.1752e-04 - val_loss: 1.8715e-04
Epoch 49/150
6444/6444 [==============================] - 3s 536us/sampl

6444/6444 [==============================] - 4s 597us/sample - loss: 2.2339e-04 - val_loss: 2.0914e-04
Epoch 62/200
6444/6444 [==============================] - 4s 563us/sample - loss: 2.2188e-04 - val_loss: 2.2433e-04
Epoch 63/200
6444/6444 [==============================] - 4s 584us/sample - loss: 2.2026e-04 - val_loss: 2.1156e-04
Epoch 64/200
6444/6444 [==============================] - 4s 588us/sample - loss: 2.2306e-04 - val_loss: 2.2002e-04
Epoch 65/200
6444/6444 [==============================] - 4s 599us/sample - loss: 2.1929e-04 - val_loss: 2.0690e-04
Epoch 66/200
6444/6444 [==============================] - 4s 569us/sample - loss: 2.1300e-04 - val_loss: 2.1462e-04
Epoch 67/200
6444/6444 [==============================] - 4s 574us/sample - loss: 2.1502e-04 - val_loss: 2.0020e-04
Epoch 68/200
6444/6444 [==============================] - 4s 587us/sample - loss: 2.1754e-04 - val_loss: 2.6253e-04
Epoch 69/200
6444/6444 [==============================] - 4s 588us/sample - loss: 2.1

6444/6444 [==============================] - 4s 626us/sample - loss: 1.5485e-04 - val_loss: 1.3608e-04
Epoch 132/200
6444/6444 [==============================] - 4s 585us/sample - loss: 1.6347e-04 - val_loss: 1.6400e-04
Validation RMSE:  0.011663374142561083
Time passed:  522.8055412769318

Deep Size:  2 , Num of Units:  45 , Learning rate:  0.0017000000000000001
Batch Size:  4 , Num of Epochs:  150
Train on 6444 samples, validate on 1612 samples
Epoch 1/150
6444/6444 [==============================] - 13s 2ms/sample - loss: 3574.5637 - val_loss: 0.3368
Epoch 2/150
6444/6444 [==============================] - 10s 2ms/sample - loss: 50.8507 - val_loss: 0.1642
Epoch 3/150
6444/6444 [==============================] - 10s 2ms/sample - loss: 369.8176 - val_loss: 0.1958
Epoch 4/150
6444/6444 [==============================] - 9s 1ms/sample - loss: 7.2782 - val_loss: 0.0790
Epoch 5/150
6444/6444 [==============================] - 9s 1ms/sample - loss: 0.8604 - val_loss: 0.0225
Epoch 6/150
644

La solución K mejor encontrada a través de GA se puede ver fácilmente usando tools.selBest(population,k = 1). Después, la configuración óptima se puede utilizar para entrenar en el conjunto de entrenamiento completo y probarlo en el conjunto de prueba de espera.

In [ ]:
# Guarda las mejores N soluciones - (1, para k=1)
best_individuals = tools.selBest(population,k = 1)
best_deep_size = None
best_num_units = None
best_learning_rate = None
best_batch_size = None
best_epochs = None

for bi in best_individuals:
    deep_size_bits = BitArray(bi[0:3])
    num_units_bits = BitArray(bi[3:8])
    learning_rate_bits = BitArray(bi[8:12])
    batch_size_bits = BitArray(bi[12:14])
    epochs_bits = BitArray(bi[14:])
    
    best_deep_size = 2*deep_size_bits.uint +2
    best_num_units = 2*num_units_bits.uint +1
    best_learning_rate = (2*learning_rate_bits.uint + 1)*10**(-4)
    best_batch_size= 2**(batch_size_bits.uint +1)
    best_epochs= (epochs_bits.uint +1)*50
    print('\nDeep Size: ', best_deep_size, ', Num of Units: ', best_num_units, ', Learning rate: ', best_learning_rate)
    print('Batch Size: ', best_batch_size, ", Num of Epochs: ", best_epochs)

In [ ]:
# Train the model using best configuration on complete training set 
#and make predictions on the test set

X,Y = prepare_dataset(train_data)
X_train, X_test, y_train, y_test = split(X, Y, test_size = 0.20, random_state = 1120)

model = keras.Sequential()
model.add(keras.Input(shape=(int(X_train.shape[1]),)))
model.add(layers.Dense(best_num_units, input_shape=(int(X_train.shape[1]),)))
#     x = LSTM(num_units, input_shape=(window_size,1))(inputs)

for i in range(best_deep_size):        
    model.add(layers.Dense(best_num_units, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

optimizer = keras.optimizers.Adam(learning_rate=best_learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-3)
model.compile(optimizer=optimizer, loss='mean_squared_error')
# callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
#                                    min_delta=1e-4,
#                                    patience=3,
#                                    restore_best_weights=True)]
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
#                     epochs=best_epochs, callbacks=callbacks, batch_size=best_batch_size, shuffle=True)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=best_epochs, batch_size=best_batch_size, shuffle=True)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE: ', rmse)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.yscale("log")
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Gráficas de comparación

In [ ]:
Y_pred = model.predict(X)

In [ ]:
plt.scatter(X[:,0], Y, s=5e0, alpha=0.7, c='red', label="Data")
plt.scatter(X[:,0], Y_pred, s=5e0, alpha=0.7, c='green', label="Model prediction")
plt.xlabel('$k$', fontsize=14); plt.ylabel(r'$A(k,r,x)$', fontsize=14)
plt.yscale('log'); plt.legend()
plt.show()

In [ ]:
plt.scatter(X[:,1], Y, s=5e0, alpha=0.7, c='red', label="Data")
plt.scatter(X[:,1], Y_pred, s=5e0, alpha=0.7, c='green', label="Model prediction")
plt.xlabel('$r$', fontsize=14); plt.ylabel(r'$A(k,r,x)$', fontsize=14)
plt.yscale('log'); plt.legend()
plt.show()

In [ ]:
plt.scatter(X[:,2], Y, s=5e0, alpha=0.7, c='red', label="Data")
plt.scatter(X[:,2], Y_pred, s=5e0, alpha=0.7, c='green', label="Model prediction")
plt.xlabel('$x$', fontsize=14); plt.ylabel(r'$A(k,r,x)$', fontsize=14)
plt.yscale('log'); plt.legend()
plt.show()

### Comparación 3D

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(X[:,0], X[:,1], X[:,2], marker='+', c=Y, cmap='seismic', alpha=1)
ax.set_xlabel('$k$', fontsize=15)
ax.set_ylabel('$r$', fontsize=15)
ax.set_zlabel('$x$', fontsize=15)
plt.colorbar(p, shrink=0.5, label='$A(k, r, x)$'); plt.title(r"Data")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(X[:,0], X[:,1], X[:,2], marker='+', c=Y_pred, cmap='seismic', alpha=1)
ax.set_xlabel('$k$', fontsize=15)
ax.set_ylabel('$r$', fontsize=15)
ax.set_zlabel('$x$', fontsize=15)
plt.colorbar(p, shrink=0.5, label='$A(k, r, x)$'); plt.title(r"Model prediction")
plt.show()